In [1]:
# Goal: manipulate qiskit programs

In [22]:
import pandas as pd
import sys
sys.path.append("../lib")
import seaborn as sns
import matplotlib.pyplot as plt
from utils import iterate_over
from utils import load_config_and_check
from inspector import Inspector
from tqdm import tqdm
import sqlite3 as sl
import os

DEBUG:matplotlib:matplotlib data path: /home/paltenmo/.local/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/paltenmo/.config/matplotlib
DEBUG:matplotlib:matplotlib version 3.4.3
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_frozen_importlib_external', '_io', 'marshal', 'posix', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', '_abc', 'abc', 'io', '_stat', 'stat', 'genericpath', 'posixpath', 'os.path', '_collections_abc', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib.abc', '_operator', 'operator', 'keyword', '_heapq', 'heapq', 'itertools', 'reprlib', '_collections', 'collections', '_functools', 

DEBUG:matplotlib:CACHEDIR=/home/paltenmo/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/paltenmo/.cache/matplotlib/fontlist-v330.json
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [23]:
%load_ext autoreload
%autoreload 2

In [4]:
# Useful additional packages
import matplotlib.pyplot as plt
import numpy as np
from math import pi

In [4]:
import qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer

In [5]:
from metamorph import get_sections
from metamorph import get_registers_used

In [6]:
QISKIT_PROGRAM_PATH = "stub_files/sample_qiskit_circuit.py"
QISKIT_PROGRAM_CONTENT = open(QISKIT_PROGRAM_PATH, 'r').read()
print(QISKIT_PROGRAM_CONTENT)

# SECTION
# NAME: PROLOGUE
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

# SECTION
# NAME: CIRCUIT
qr_0 = QuantumRegister(2)
c_0 = QuantumCircuit(qr_0)

c_0.h(qr_0[0])
c_0.crz(1, qr_0[0], qr_0[1])
c_0.barrier()
c_0.id(qr_0[1])
c_0.u(1, 2, -2, qr_0[0])
c_0.u(2, 3, -2, qr_0[0])
c_0.u(3, 4, -2, qr_0[0])


# SECTION
# NAME: OPTIMIZATION

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGates

passmanager = PassManager()
passmanager.append(Optimize1qGates())


# SECTION
# NAME: EXECUTION
from qiskit import Aer, transpile, execute

shots = 8192

c_0.measure_all()

backend = Aer.get_backend('qasm_simulator')
job = execute(c_0, backend=backend, shots=shots)
job_result = job.result()
count = job_result.get_counts(c_0)
print(count)

RESULT = count


# Convert to a gate and stick it into an arbitrary place in the bigger circuit
#sub_inst = sub_circ.to_instruction()

#qr = QuantumRegister(3, 'q')
#circ = QuantumCircuit(qr)
#circ.h(qr

In [7]:
sections = get_sections(QISKIT_PROGRAM_CONTENT)
circuit_code = sections["CIRCUIT"]

In [8]:
import ast
import astpretty

In [9]:
tree = ast.parse(circuit_code)

In [10]:
astpretty.pprint(tree)

Module(
    body=[
        Assign(
            lineno=2,
            col_offset=0,
            end_lineno=2,
            end_col_offset=25,
            targets=[Name(lineno=2, col_offset=0, end_lineno=2, end_col_offset=4, id='qr_0', ctx=Store())],
            value=Call(
                lineno=2,
                col_offset=7,
                end_lineno=2,
                end_col_offset=25,
                func=Name(lineno=2, col_offset=7, end_lineno=2, end_col_offset=22, id='QuantumRegister', ctx=Load()),
                args=[Constant(lineno=2, col_offset=23, end_lineno=2, end_col_offset=24, value=2, kind=None)],
                keywords=[],
            ),
            type_comment=None,
        ),
        Assign(
            lineno=3,
            col_offset=0,
            end_lineno=3,
            end_col_offset=26,
            targets=[Name(lineno=3, col_offset=0, end_lineno=3, end_col_offset=3, id='c_0', ctx=Store())],
            value=Call(
                lineno=3,
              

## Generate a Circuit from a Qiskit algorithm

In [50]:
from qiskit.algorithms import AmplitudeEstimation
from qiskit.algorithms import EstimationProblem

In [54]:
class BernoulliStateIn(QuantumCircuit):
    """A circuit preparing sqrt(1 - p)|0> + sqrt(p)|1>."""

    def __init__(self, probability):
        super().__init__(1)
        angle = 2 * np.arcsin(np.sqrt(probability))
        self.ry(angle, 0)

prob = 0.8
a_op = BernoulliStateIn(prob)
problem = EstimationProblem(a_op, objective_qubits=0)
qc = AmplitudeEstimation(4).construct_circuit(estimation_problem=problem)

In [56]:
qc.draw()

┌──────┐
eval_0: ──────────────┤0     ├
                      │      │
eval_1: ──────────────┤1     ├
                      │      │
eval_2: ──────────────┤2 QPE ├
                      │      │
eval_3: ──────────────┤3     ├
        ┌────────────┐│      │
     q: ┤ Ry(2.2143) ├┤4     ├
        └────────────┘└──────┘

In [69]:
qasm_conversion = qc.qasm()
qasm_conversion = qasm_conversion.replace(";", ";\n")
print(qasm_conversion)

OPENQASM 2.0;

include "qelib1.inc";

gate gate_IQFT q0,q1,q2,q3 { h q0;
 cp(-pi/2) q1,q0;
 h q1;
 cp(-pi/4) q2,q0;
 cp(-pi/2) q2,q1;
 h q2;
 cp(-pi/8) q3,q0;
 cp(-pi/4) q3,q1;
 cp(-pi/2) q3,q2;
 h q3;
 }
gate cQ__8 q0,q1 { cu(pi/2,0,pi,0) q0,q1;
 cx q0,q1;
 cu(pi/2,0,pi,0) q0,q1;
 p(0) q0;
 p(0) q1;
 cx q0,q1;
 u(1.1071487,0,0) q1;
 cx q0,q1;
 u(-1.1071487,0,0) q1;
 cx q0,q1;
 h q1;
 cx q0,q1;
 h q1;
 cx q0,q1;
 p(0) q0;
 p(0) q1;
 cx q0,q1;
 u(-1.1071487,0,0) q1;
 cx q0,q1;
 u(1.1071487,0,0) q1;
 cu(pi/2,0,pi,0) q0,q1;
 cx q0,q1;
 cu(pi/2,0,pi,0) q0,q1;
 p(0) q0;
 p(0) q1;
 cx q0,q1;
 u(1.1071487,0,0) q1;
 cx q0,q1;
 u(-1.1071487,0,0) q1;
 cx q0,q1;
 h q1;
 cx q0,q1;
 h q1;
 cx q0,q1;
 p(0) q0;
 p(0) q1;
 cx q0,q1;
 u(-1.1071487,0,0) q1;
 cx q0,q1;
 u(1.1071487,0,0) q1;
 cu(pi/2,0,pi,0) q0,q1;
 cx q0,q1;
 cu(pi/2,0,pi,0) q0,q1;
 p(0) q0;
 p(0) q1;
 cx q0,q1;
 u(1.1071487,0,0) q1;
 cx q0,q1;
 u(-1.1071487,0,0) q1;
 cx q0,q1;
 h q1;
 cx q0,q1;
 h q1;
 cx q0,q1;
 p(0) q0;
 p(0) q1;
 cx 

In [58]:
qc.decompose().draw()

┌───┐                                      ┌───────┐
eval_0: ──────┤ H ├─────────■────────────────────────────┤3      ├
              ├───┤         │                            │       │
eval_1: ──────┤ H ├─────────┼───────■────────────────────┤2      ├
              ├───┤         │       │                    │  IQFT │
eval_2: ──────┤ H ├─────────┼───────┼───────■────────────┤1      ├
              ├───┤         │       │       │            │       │
eval_3: ──────┤ H ├─────────┼───────┼───────┼───────■────┤0      ├
        ┌─────┴───┴─────┐┌──┴───┐┌──┴───┐┌──┴───┐┌──┴───┐└───────┘
     q: ┤ R(2.2143,π/2) ├┤ Q**1 ├┤ Q**2 ├┤ Q**4 ├┤ Q**8 ├─────────
        └───────────────┘└──────┘└──────┘└──────┘└──────┘

In [64]:
qc.decompose().decompose().draw()

┌─────────┐                                         ┌──────┐     »
eval_0: ───┤ U2(0,π) ├───────────■──────────■─────────■────────┤ P(0) ├──■──»
           ├─────────┤           │          │         │        └──────┘  │  »
eval_1: ───┤ U2(0,π) ├───────────┼──────────┼─────────┼──────────────────┼──»
           ├─────────┤           │          │         │                  │  »
eval_2: ───┤ U2(0,π) ├───────────┼──────────┼─────────┼──────────────────┼──»
           ├─────────┤           │          │         │                  │  »
eval_3: ───┤ U2(0,π) ├───────────┼──────────┼─────────┼──────────────────┼──»
        ┌──┴─────────┴───┐┌──────┴───────┐┌─┴─┐┌──────┴───────┐┌──────┐┌─┴─┐»
     q: ┤ U3(2.2143,0,0) ├┤ U(π/2,0,π,0) ├┤ X ├┤ U(π/2,0,π,0) ├┤ P(0) ├┤ X ├»
        └────────────────┘└──────────────┘└───┘└──────────────┘└──────┘└───┘»
«                                                                         »
«eval_0: ───────────────────■──────────────────────■─────────■─────────■──»
«                           │                      │         │         │  »
«eval_1: ───────────────────┼──────────────────────┼─────────┼─────────┼──»
«                           │                      │         │         │  »
«eval_2: ───────────────────┼──────────────────────┼─────────┼─────────┼──»
«                           │                      │         │         │  »
«eval_3: ───────────────────┼──────────────────────┼─────────┼─────────┼──»
«        ┌───────────────┐┌─┴─┐┌────────────────┐┌─┴─┐┌───┐┌─┴─┐┌───┐┌─┴─┐»
«     q: ┤ U(1.1071,0,0) ├┤ X ├┤ U(-1.1071,0,0) ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├»
«        └───────────────┘└───┘└────────────────┘└───┘└───┘└───┘└───┘└───┘»
«        ┌──────┐                                 ┌──────┐                    »
«eval_0: ┤ P(0) ├──■──────────────────────■───────┤ P(π) ├────────────────────»
«        └──────┘  │                      │       └──────┘                    »
«eval_1: ──────────┼──────────────────────┼──────────────────────────■────────»
«                  │                      │                          │        »
«eval_2: ──────────┼──────────────────────┼──────────────────────────┼────────»
«                  │                      │                          │        »
«eval_3: ──────────┼──────────────────────┼──────────────────────────┼────────»
«        ┌──────┐┌─┴─┐┌────────────────┐┌─┴─┐┌───────────────┐┌──────┴───────┐»
«     q: ┤ P(0) ├┤ X ├┤ U(-1.1071,0,0) ├┤ X ├┤ U(1.1071,0,0) ├┤ U(π/2,0,π,0) ├»
«        └──────┘└───┘└────────────────┘└───┘└───────────────┘└──────────────┘»
«                                                                »
«eval_0: ────────────────────────────────────────────────────────»
«                             ┌──────┐                           »
«eval_1: ──■─────────■────────┤ P(0) ├──■─────────────────────■──»
«          │         │        └──────┘  │                     │  »
«eval_2: ──┼─────────┼──────────────────┼─────────────────────┼──»
«          │         │                  │                     │  »
«eval_3: ──┼─────────┼──────────────────┼─────────────────────┼──»
«        ┌─┴─┐┌──────┴───────┐┌──────┐┌─┴─┐┌───────────────┐┌─┴─┐»
«     q: ┤ X ├┤ U(π/2,0,π,0) ├┤ P(0) ├┤ X ├┤ U(1.1071,0,0) ├┤ X ├»
«        └───┘└──────────────┘└──────┘└───┘└───────────────┘└───┘»
«                                                                »
«eval_0: ────────────────────────────────────────────────────────»
«                                                   ┌──────┐     »
«eval_1: ────────────────────■─────────■─────────■──┤ P(0) ├──■──»
«                            │         │         │  └──────┘  │  »
«eval_2: ────────────────────┼─────────┼─────────┼────────────┼──»
«                            │         │         │            │  »
«eval_3: ────────────────────┼─────────┼─────────┼────────────┼──»
«        ┌────────────────┐┌─┴─┐┌───┐┌─┴─┐┌───┐┌─┴─┐┌──────┐┌─┴─┐»
«     q: ┤ U(-1.1071,0,0) ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ P(0) ├┤ X ├»
«        └────────────────┘└──

In [70]:
qc_decomposed = qc.decompose().decompose().decompose().decompose().decompose().decompose().decompose()
qasm_conversion = qc_decomposed.qasm()
qasm_conversion = qasm_conversion.replace(";", ";\n")
print(qasm_conversion)

OPENQASM 2.0;

include "qelib1.inc";

qreg eval[4];

qreg q[1];

u(pi/2,0,pi) eval[0];

u(0,0,0) eval[0];

u(0,0,pi/2) eval[0];

u(pi/2,0,pi) eval[1];

u(0,0,0) eval[1];

u(0,0,pi/2) eval[1];

u(pi/2,0,pi) eval[2];

u(0,0,0) eval[2];

u(0,0,pi/2) eval[2];

u(pi/2,0,pi) eval[3];

u(0,0,0) eval[3];

u(0,0,pi/2) eval[3];

u(2.2142974,0,0) q[0];

u(0,0,pi/2) q[0];

cx eval[0],q[0];

u(-pi/4,0,-pi/2) q[0];

cx eval[0],q[0];

u(pi/4,0,0) q[0];

cx eval[0],q[0];

u(0,0,pi/2) q[0];

u(0,0,0) eval[0];

u(0,0,pi/2) eval[0];

cx eval[0],q[0];

u(-pi/4,0,-pi/2) q[0];

cx eval[0],q[0];

u(pi/4,0,0) q[0];

u(0,0,0) q[0];

u(0,0,0) eval[0];

cx eval[0],q[0];

u(1.1071487,0,0) q[0];

cx eval[0],q[0];

u(-1.1071487,0,0) q[0];

cx eval[0],q[0];

u(pi/2,0,pi) q[0];

cx eval[0],q[0];

u(pi/2,0,pi) q[0];

cx eval[0],q[0];

u(0,0,0) q[0];

u(0,0,0) eval[0];

cx eval[0],q[0];

u(-1.1071487,0,0) q[0];

cx eval[0],q[0];

u(1.1071487,0,0) q[0];

u(0,0,pi/2) q[0];

u(0,0,pi) eval[0];

u(0,0,-pi/16) eval[0];

cx 

## Read Registers from AST

In [11]:
registers = get_registers_used(circ_definition=circuit_code)
registers

[{'name': 'qr_0', 'type': 'QuantumRegister', 'size': 2}]

In [94]:
GLOBALS = {"RESULT": 0}

exec(QISKIT_PROGRAM_CONTENT, GLOBALS)

{'00': 3058, '01': 5134}


In [96]:
GLOBALS["RESULT"]

{'00': 3058, '01': 5134}

## Auto Generation of Qiskit Code - Circuit

In [36]:
from generation_strategy_python import QiskitFuzzer

In [112]:
config = load_config_and_check("../config/qmt_v00.yaml")

In [90]:
fuzzer = QiskitFuzzer()

In [46]:
source_code, metadata = fuzzer.generate_via_atomic_ops(
    gate_set=config["generation_strategy"]["gate_set"],
    n_qubits=5,
    n_ops=10
)

In [53]:
[b.name for b in backends]

[<bound method AerSimulator.name of AerSimulator('aer_simulator')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_statevector')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_statevector_gpu')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_density_matrix')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_density_matrix_gpu')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_stabilizer')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_matrix_product_state')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_extended_stabilizer')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_unitary')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_unitary_gpu')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_superop')>,
 <bound method BackendV1.name of QasmSimulator('qasm_simulator')>,
 <bound method BackendV1.name of Statevecto

In [104]:
long_str = """[<bound method AerSimulator.name of AerSimulator('aer_simulator')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_statevector')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_statevector_gpu')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_density_matrix')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_density_matrix_gpu')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_stabilizer')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_matrix_product_state')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_extended_stabilizer')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_unitary')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_unitary_gpu')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_superop')>,
 <bound method BackendV1.name of QasmSimulator('qasm_simulator')>,
 <bound method BackendV1.name of StatevectorSimulator('statevector_simulator')>,
 <bound method BackendV1.name of UnitarySimulator('unitary_simulator')>,
 <bound method BackendV1.name of PulseSimulator('pulse_simulator')>]"""

for line in long_str.split("\n"):
    print("- " + line.split("('")[1].replace("')>", "").replace(",", ""))

- aer_simulator
- aer_simulator_statevector
- aer_simulator_statevector_gpu
- aer_simulator_density_matrix
- aer_simulator_density_matrix_gpu
- aer_simulator_stabilizer
- aer_simulator_matrix_product_state
- aer_simulator_extended_stabilizer
- aer_simulator_unitary
- aer_simulator_unitary_gpu
- aer_simulator_superop
- qasm_simulator
- statevector_simulator
- unitary_simulator
- pulse_simulator]


In [106]:
type(np.random.choice([1,2,3]))

numpy.int64

In [115]:
generated_py_file, metadata = fuzzer.generate_file(
    gate_set=config["generation_strategy"]["gate_set"],
    n_qubits=5,
    n_ops=10,
    optimizations=["CommutationAnalysis", "CommutativeCancellation", "CXCancellation"],
    backend="qasm_simulator",
    shots=8192)
print(generated_py_file)


# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *

# SECTION
# NAME: CIRCUIT

qr_qr = QuantumRegister(5, name='qr_qr')
cr_qr = ClassicalRegister(5, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(CSwapGate(), qargs=[qr_qr[3], qr_qr[0], qr_qr[1]], cargs=[])
qc.append(XGate(), qargs=[qr_qr[0]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[3]], cargs=[])
qc.append(CHGate(), qargs=[qr_qr[4], qr_qr[0]], cargs=[])
qc.append(RZZGate(3.381309563488947), qargs=[qr_qr[4], qr_qr[1]], cargs=[])
qc.append(U2Gate(3.4690789366725516,6.089828077752466), qargs=[qr_qr[2]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[1]], cargs=[])
qc.append(RZXGate(4.008916676545142), qargs=[qr_qr[2], qr_qr[1]], cargs=[])
qc.append(RC3XGate(), qargs=[qr_qr[2], qr_qr[0], qr_qr[1], qr_qr[3]], cargs=[])
qc.append(RCCXGate(), qargs=[qr_qr[1], qr_qr[0], qr_qr[2]], cargs=

In [113]:
from metamorph import change_backend

In [142]:
modified_code = change_backend(source_code=generated_py_file, available_backends=config["generation_strategy"]["backends"])

#astpretty.pprint(modified_code)

Found: qasm_simulator


In [143]:
print(modified_code)

# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *
# SECTION
# NAME: CIRCUIT

qr_qr = QuantumRegister(5, name='qr_qr')
cr_qr = ClassicalRegister(5, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(CSwapGate(), qargs=[qr_qr[3], qr_qr[0], qr_qr[1]], cargs=[])
qc.append(XGate(), qargs=[qr_qr[0]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[3]], cargs=[])
qc.append(CHGate(), qargs=[qr_qr[4], qr_qr[0]], cargs=[])
qc.append(RZZGate(3.381309563488947), qargs=[qr_qr[4], qr_qr[1]], cargs=[])
qc.append(U2Gate(3.4690789366725516, 6.089828077752466), qargs=[qr_qr[2]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[1]], cargs=[])
qc.append(RZXGate(4.008916676545142), qargs=[qr_qr[2], qr_qr[1]], cargs=[])
qc.append(RC3XGate(), qargs=[qr_qr[2], qr_qr[0], qr_qr[1], qr_qr[3]], cargs=[])
qc.append(RCCXGate(), qargs=[qr_qr[1], qr_qr[0], qr_qr[2]], cargs=[

In [137]:
!pip install astor

  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)


In [124]:
print(modified_code)

# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *
# SECTION
# NAME: CIRCUIT

qr_qr = QuantumRegister(5, name='qr_qr')
cr_qr = ClassicalRegister(5, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(CSwapGate(), qargs=[qr_qr[3], qr_qr[0], qr_qr[1]], cargs=[])
qc.append(XGate(), qargs=[qr_qr[0]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[3]], cargs=[])
qc.append(CHGate(), qargs=[qr_qr[4], qr_qr[0]], cargs=[])
qc.append(RZZGate(3.381309563488947), qargs=[qr_qr[4], qr_qr[1]], cargs=[])
qc.append(U2Gate(3.4690789366725516, 6.089828077752466), qargs=[qr_qr[2]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[1]], cargs=[])
qc.append(RZXGate(4.008916676545142), qargs=[qr_qr[2], qr_qr[1]], cargs=[])
qc.append(RC3XGate(), qargs=[qr_qr[2], qr_qr[0], qr_qr[1], qr_qr[3]], cargs=[])
qc.append(RCCXGate(), qargs=[qr_qr[1], qr_qr[0], qr_qr[2]], cargs=[

In [111]:
astpretty.pprint(ast.parse('Aer.get_backend("qasm_simulator")', mode='eval'), indent=4)

Expression(
    body=Call(
        lineno=1,
        col_offset=0,
        end_lineno=1,
        end_col_offset=33,
        func=Attribute(
            lineno=1,
            col_offset=0,
            end_lineno=1,
            end_col_offset=15,
            value=Name(lineno=1, col_offset=0, end_lineno=1, end_col_offset=3, id='Aer', ctx=Load()),
            attr='get_backend',
            ctx=Load(),
        ),
        args=[Constant(lineno=1, col_offset=16, end_lineno=1, end_col_offset=32, value='qasm_simulator', kind=None)],
        keywords=[],
    ),
)


In [101]:
GLOBALS = {"RESULT": 0}

exec(generated_py_file, GLOBALS)

In [102]:
GLOBALS["RESULT"]

{'11000': 10,
 '00100': 3853,
 '01100': 3609,
 '10000': 5,
 '10100': 178,
 '11100': 195,
 '01000': 154,
 '00000': 188}

In [ ]:
qr_qr = QuantumRegister(5, name='qr_qr')
cr_qr = ClassicalRegister(5, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(CZGate(), qargs=[qr_qr[4], qr_qr[3]], cargs=[])
qc.append(IGate(), qargs=[qr_qr[1]], cargs=[])
qc.append(C3SXGate(), qargs=[qr_qr[3], qr_qr[0], qr_qr[2]], cargs=[])
qc.append(RXGate(2.700932295903693), qargs=[qr_qr[0]], cargs=[])
qc.append(U3Gate(0.09144240096955454,5.910826577430733,1.3545764490066696), qargs=[qr_qr[3]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[4], qr_qr[1]], cargs=[])
qc.append(SGate(), qargs=[qr_qr[0]], cargs=[])
qc.append(C3SXGate(), qargs=[qr_qr[3], qr_qr[0], qr_qr[1]], cargs=[])
qc.append(RYYGate(1.3519467402030274), qargs=[qr_qr[1], qr_qr[3]], cargs=[])
qc.append(RCCXGate(), qargs=[qr_qr[0], qr_qr[1], qr_qr[3]], cargs=[])

---------------------------------------------------------------------------
CircuitError                              Traceback (most recent call last)
/tmp/ipykernel_33593/513803224.py in <module>
      1 GLOBALS = {"RESULT": 0}
      2 
----> 3 exec(generated_py_file, GLOBALS)

<string> in <module>

~/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py in append(self, instruction, qargs, cargs)
   1228             requester = self._resolve_classical_resource
   1229         instructions = InstructionSet(resource_requester=requester)
-> 1230         for qarg, carg in instruction.broadcast_arguments(expanded_qargs, expanded_cargs):
   1231             instructions.add(appender(instruction, qarg, carg), qarg, carg)
   1232         return instructions

~/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/circuit/gate.py in broadcast_arguments(self, qargs, cargs)
    198         """
    199         if len(qargs) != self.num_qubits or cargs:
--> 200             raise CircuitError(
    201                 f"The amount of qubit({len(qargs)})/clbit({len(cargs)}) arguments does"
    202                 f" not match the gate expectation ({self.num_qubits})."

CircuitError: 'The amount of qubit(3)/clbit(0) arguments does not match the gate expectation (4).'

In [145]:
# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *

# SECTION
# NAME: CIRCUIT

qr_qr = QuantumRegister(13, name='qr_qr')
cr_qr = ClassicalRegister(13, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(PhaseGate(3.9609333943213456), qargs=[qr_qr[9]], cargs=[])
qc.draw(fold=-1)

qr_qr_0: ─────────────
                       
 qr_qr_1: ─────────────
                       
 qr_qr_2: ─────────────
                       
 qr_qr_3: ─────────────
                       
 qr_qr_4: ─────────────
                       
 qr_qr_5: ─────────────
                       
 qr_qr_6: ─────────────
                       
 qr_qr_7: ─────────────
                       
 qr_qr_8: ─────────────
          ┌───────────┐
 qr_qr_9: ┤ P(3.9609) ├
          └───────────┘
qr_qr_10: ─────────────
                       
qr_qr_11: ─────────────
                       
qr_qr_12: ─────────────
                       
cr_qr: 13/═════════════

In [ ]:
# SECTION
# NAME: OPTIMIZATION

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import *
passmanager = PassManager()
passmanager.append(RemoveDiagonalGatesBeforeMeasure())
passmanager.append(OptimizeSwapBeforeMeasure())
passmanager.append(Collect2qBlocks())
qc = passmanager.run(qc)

In [146]:
# SECTION
# NAME: MEASUREMENT

qc.measure(qr_qr, cr_qr)
qc.draw(fold=-1)

┌─┐                                 
 qr_qr_0: ─────────────┤M├─────────────────────────────────
                       └╥┘┌─┐                              
 qr_qr_1: ──────────────╫─┤M├──────────────────────────────
                        ║ └╥┘┌─┐                           
 qr_qr_2: ──────────────╫──╫─┤M├───────────────────────────
                        ║  ║ └╥┘┌─┐                        
 qr_qr_3: ──────────────╫──╫──╫─┤M├────────────────────────
                        ║  ║  ║ └╥┘┌─┐                     
 qr_qr_4: ──────────────╫──╫──╫──╫─┤M├─────────────────────
                        ║  ║  ║  ║ └╥┘┌─┐                  
 qr_qr_5: ──────────────╫──╫──╫──╫──╫─┤M├──────────────────
                        ║  ║  ║  ║  ║ └╥┘┌─┐               
 qr_qr_6: ──────────────╫──╫──╫──╫──╫──╫─┤M├───────────────
                        ║  ║  ║  ║  ║  ║ └╥┘┌─┐            
 qr_qr_7: ──────────────╫──╫──╫──╫──╫──╫──╫─┤M├────────────
                        ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         
 qr_qr_8: ──────────────╫──╫──╫──╫──╫──╫──╫──╫─┤M├─────────
          ┌───────────┐ ║  ║  ║  ║  ║  ║  ║  ║ └╥┘      ┌─┐
 qr_qr_9: ┤ P(3.9609) ├─╫──╫──╫──╫──╫──╫──╫──╫──╫───────┤M├
          └────┬─┬────┘ ║  ║  ║  ║  ║  ║  ║  ║  ║       └╥┘
qr_qr_10: ─────┤M├──────╫──╫──╫──╫──╫──╫──╫──╫──╫────────╫─
               └╥┘      ║  ║  ║  ║  ║  ║  ║  ║  ║ ┌─┐    ║ 
qr_qr_11: ──────╫───────╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├────╫─
                ║       ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐ ║ 
qr_qr_12: ──────╫───────╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├─╫─
                ║       ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘ ║ 
cr_qr: 13/══════╩═══════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                10      0  1  2  3  4  5  6  7  8  11 12 9

In [147]:
# SECTION
# NAME: EXECUTION

from qiskit import Aer, transpile, execute
backend_0e3b482c4dd240f5bb9a897a581724c3 = Aer.get_backend('aer_simulator_statevector_gpu')
RESULT = execute(qc, backend=backend_0e3b482c4dd240f5bb9a897a581724c3, shots=15676).result().get_counts(qc)

In [148]:
RESULT

{'0000000000000': 15676}

In [150]:
# SECTION
# NAME: EXECUTION
from qiskit import Aer, transpile, execute
backend_0e3b482c4dd240f5bb9a897a581724c3 = Aer.get_backend('aer_simulator_superop')
RESULT = execute(qc, backend=backend_0e3b482c4dd240f5bb9a897a581724c3,
    shots=15676).result().get_counts(qc)

Simulation failed and returned the following error message:
ERROR:  [Experiment 0] Circuit qc contains invalid instructions {"instructions": {measure}} for "superop" method.


QiskitError: 'Data for experiment "qc" could not be found.'

In [158]:

# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *

# SECTION
# NAME: CIRCUIT

qr_qr = QuantumRegister(10, name='qr_qr')
cr_qr = ClassicalRegister(10, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(CZGate(), qargs=[qr_qr[4], qr_qr[6]], cargs=[])

# SECTION
# NAME: OPTIMIZATION

# SECTION
# NAME: MEASUREMENT

qc.measure(qr_qr, cr_qr)

# SECTION
# NAME: EXECUTION

from qiskit import Aer, transpile, execute
backend_39f225015fb547da844865a411b8f67a = Aer.get_backend('aer_simulator_unitary')
job_result = execute(qc, backend=backend_39f225015fb547da844865a411b8f67a).result().get_unitary()
print(job_result)
#RESULT = job_result..get_unitary(qc)

Simulation failed and returned the following error message:
ERROR:  [Experiment 0] Circuit qc contains invalid instructions {"instructions": {measure}} for "unitary" method.


QiskitError: 'ERROR:  [Experiment 0] Circuit qc contains invalid instructions {"instructions": {measure}} for "unitary" method. ,  ERROR: Circuit qc contains invalid instructions {"instructions": {measure}} for "unitary" method.'

## Check Exceptions raised

In [5]:
PATH = "../data/qmt_v00/programs/metadata"

In [6]:
from utils import iterate_over

In [10]:
exec_files = [os.path.join(PATH, f) for f in os.listdir(PATH) if f.endswith("_exec.json")]
exec_files

['../data/qmt_v00/programs/metadata/e2b17692617344dfb40483e16c9d8941_exec.json',
 '../data/qmt_v00/programs/metadata/b71ac6eaafd74892b2ba50b2a1883ffd_exec.json',
 '../data/qmt_v00/programs/metadata/c4fcb9af981545218ae509060981ae93_exec.json',
 '../data/qmt_v00/programs/metadata/44d530acc9b549a0958c2e0bb8ac7551_exec.json',
 '../data/qmt_v00/programs/metadata/a19ef055a1ad4cbdbd0219dc2c48aee3_exec.json',
 '../data/qmt_v00/programs/metadata/19c0a104b46d49a9803ba1177e8cb094_exec.json',
 '../data/qmt_v00/programs/metadata/4bbea17616f346d79e09c3fd1d28ac6c_exec.json',
 '../data/qmt_v00/programs/metadata/56a8388ac978494e8d2b0a29c2313908_exec.json',
 '../data/qmt_v00/programs/metadata/bda38b6075fd4850864ce6cc42564f6c_exec.json',
 '../data/qmt_v00/programs/metadata/9ca25d6933ac4347b69677e63e45c302_exec.json',
 '../data/qmt_v00/programs/metadata/7b6d4a69dbef4339abdb866b46ee1453_exec.json',
 '../data/qmt_v00/programs/metadata/e480d321b48c479783b1fa442ae168d3_exec.json',
 '../data/qmt_v00/programs/m

In [23]:
records = []
for program_id, exec_dict in iterate_over(PATH, filetype=".json", parse_json=True):
    print(program_id)
    print(type(exec_dict))
    
    new_record = dict(exec_dict)
    new_record["program_id"] = program_id
    records.append(new_record)

34ba87454307498eb8ee1a6aa9dd8358
<class 'dict'>
e2b17692617344dfb40483e16c9d8941_exec
<class 'dict'>
0bcb88b2c24442b2805897ad15dc14b8
<class 'dict'>
b71ac6eaafd74892b2ba50b2a1883ffd_exec
<class 'dict'>
6cedc0b5235a447aa963266d9bd387e4
<class 'dict'>
c4fcb9af981545218ae509060981ae93_exec
<class 'dict'>
76141533766f4ad58505328a0f7700c9
<class 'dict'>
1da97528a4c646e6af2e62201a6ee8c6
<class 'dict'>
8cb689d5d2214d4d8e4e0ba295c97bce
<class 'dict'>
44d530acc9b549a0958c2e0bb8ac7551_exec
<class 'dict'>
629ad3d6d41d4c8c8ad28b0adedf9e84
<class 'dict'>
a19ef055a1ad4cbdbd0219dc2c48aee3_exec
<class 'dict'>
206a812c53db4376b7b5876bb59a64c4
<class 'dict'>
19c0a104b46d49a9803ba1177e8cb094_exec
<class 'dict'>
6b245fee8bb64a2b8d1d1fc93f365576
<class 'dict'>
4bbea17616f346d79e09c3fd1d28ac6c_exec
<class 'dict'>
56a8388ac978494e8d2b0a29c2313908_exec
<class 'dict'>
0834361d31df499eaaa57e48e60dd0db
<class 'dict'>
544bfc5c3c534d95b55f7da7b0abe716
<class 'dict'>
bda38b6075fd4850864ce6cc42564f6c_exec
<class 'di

5b4336f67e78442db69a5aaf356185dc
<class 'dict'>
3237479a476e4479817425d0d7e36186
<class 'dict'>
302aa226d61d41dfa302e4b049f98ba0
<class 'dict'>
def7266e366d49f7802dfec20e9c3066_exec
<class 'dict'>
5b4336f67e78442db69a5aaf356185dc_exec
<class 'dict'>
6242f600990f464fb17214fc5ee31e37
<class 'dict'>
cbbb1795a304435c9cbb0117dfa8ed32
<class 'dict'>
919ff7f85f804be4a3135d0b92e36d53
<class 'dict'>
bb7966432be64c0f93b9ec252f8439f9_exec
<class 'dict'>
0e7d7f29604b40cd84bf437e0769955c_exec
<class 'dict'>
545ffa35c6ce496e9852ff4671ae2325
<class 'dict'>
44d530acc9b549a0958c2e0bb8ac7551
<class 'dict'>
7ad78bef812c4e0c87936b6d29881c2f_exec
<class 'dict'>
1f131498b0104d47a047087632bd5751_exec
<class 'dict'>
7b6d4a69dbef4339abdb866b46ee1453
<class 'dict'>
dcd7e5977cc14d13b93dddab531d323f_exec
<class 'dict'>
e480d321b48c479783b1fa442ae168d3
<class 'dict'>
0900b3b660c74e0abcfd57e3711a7487_exec
<class 'dict'>
b29769e655594fe7895e7a964ecdfa4f_exec
<class 'dict'>
19c0a104b46d49a9803ba1177e8cb094
<class 'di

JSONDecodeError: Expecting value: line 6 column 24 (char 160)

## Manual Benchmark

In [24]:
import sqlite3 as sl

In [31]:
con = sl.connect("../data/qmt_v00/qfl.db")

In [32]:
df = pd.read_sql("""SELECT * FROM QFLDATA""", con)

In [33]:
df.head()

,index,program_id,source.program_id,source.selected_gate_set,source.selected_optimization,source.shots,source.n_qubits,source.n_ops,source.py_file_path,source.circuit_id,...,followup.n_ops,followup.py_file_path,followup.circuit_id,followup.id_quantum_reg,followup.id_classical_reg,followup.gate_set,divergence.ks.statistic,divergence.ks.p-value,exceptions.source,exceptions.followup
0,0,5ea834eb269c45bab5c5ce6976745a5e,5ea834eb269c45bab5c5ce6976745a5e,"['RXGate', 'CSXGate', 'U1Gate', 'U2Gate', 'RZZ...","['CommutationAnalysis', 'CrosstalkAdaptiveSche...",244,1,29,../data/qmt_v00/programs/source/5ea834eb269c45...,qc,...,29,../data/qmt_v00/programs/followup/5ea834eb269c...,qc,qr_qr,cr_qr,"[{'name': 'RXGate', 'n_bits': 1, 'n_params': 1...",0.0,1.0,__init__() missing 2 required positional argum...,__init__() missing 2 required positional argum...
1,0,f5be7ea9cf254aca9363e1398b51048b,f5be7ea9cf254aca9363e1398b51048b,"['CXGate', 'IGate', 'PhaseGate', 'U1Gate', 'HG...","['OptimizeSwapBeforeMeasure', 'Optimize1qGates...",177360,20,11,../data/qmt_v00/programs/source/f5be7ea9cf254a...,qc,...,11,../data/qmt_v00/programs/followup/f5be7ea9cf25...,qc,qr_qr,cr_qr,"[{'name': 'CXGate', 'n_bits': 2, 'n_params': 0...",0.0,1.0,'NoneType' object has no attribute 'items','NoneType' object has no attribute 'items'
2,0,6323411283d342f7a81776945bd080ec,6323411283d342f7a81776945bd080ec,"['SXGate', 'RZZGate', 'CU1Gate', 'CYGate', 'CX...","['CrosstalkAdaptiveSchedule', 'CollectMultiQBl...",44340,16,12,../data/qmt_v00/programs/source/6323411283d342...,qc,...,12,../data/qmt_v00/programs/followup/6323411283d3...,qc,qr_qr,cr_qr,"[{'name': 'SXGate', 'n_bits': 1, 'n_params': 0...",0.0,1.0,'The amount of qubit(3)/clbit(0) arguments doe...,'The amount of qubit(3)/clbit(0) arguments doe...
3,0,47b0a71f4f584b11ad90a2e4a4b511b8,47b0a71f4f584b11ad90a2e4a4b511b8,"['C4XGate', 'TGate', 'RCCXGate', 'CSXGate', 'R...","['HoareOptimizer', 'OptimizeSwapBeforeMeasure'...",979,5,28,../data/qmt_v00/programs/source/47b0a71f4f584b...,qc,...,28,../data/qmt_v00/programs/followup/47b0a71f4f58...,qc,qr_qr,cr_qr,"[{'name': 'C4XGate', 'n_bits': 4, 'n_params': ...",0.0,1.0,'z3-solver is required to use HoareOptimizer. ...,'z3-solver is required to use HoareOptimizer. ...
4,0,0426748ff8304cc3850a12344637b21c,0426748ff8304cc3850a12344637b21c,"['CU1Gate', 'SwapGate', 'ECRGate', 'CXGate', '...","['InverseCancellation', 'EchoRZXWeylDecomposit...",692,4,20,../data/qmt_v00/programs/source/0426748ff8304c...,qc,...,20,../data/qmt_v00/programs/followup/0426748ff830...,qc,qr_qr,cr_qr,"[{'name': 'CU1Gate', 'n_bits': 2, 'n_params': ...",0.0,1.0,'The amount of qubit(4)/clbit(0) arguments doe...,'The amount of qubit(4)/clbit(0) arguments doe...


In [36]:
iterate_over

{}

In [4]:
# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *
# SECTION
# NAME: CIRCUIT

qr_qr = QuantumRegister(16, name='qr_qr')
cr_qr = ClassicalRegister(16, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(C3XGate(3.907524998275438), qargs=[qr_qr[11], qr_qr[8], qr_qr[7], qr_qr[0]], cargs=[])
qc.append(RC3XGate(), qargs=[qr_qr[8], qr_qr[15], qr_qr[5], qr_qr[14]], cargs=[])
qc.append(CHGate(), qargs=[qr_qr[10], qr_qr[15]], cargs=[])
qc.append(CU1Gate(4.818142106647685), qargs=[qr_qr[0], qr_qr[6]], cargs=[])
qc.append(RYGate(1.0309627158294603), qargs=[qr_qr[4]], cargs=[])
qc.append(CU1Gate(5.752429763184833), qargs=[qr_qr[8], qr_qr[12]], cargs=[])
qc.append(CRZGate(2.729319301936734), qargs=[qr_qr[15], qr_qr[7]], cargs=[])
qc.append(CUGate(1.7865204421912357, 2.217990729902729, 5.83693248435634, 2.9917879555121014), qargs=[qr_qr[1], qr_qr[0]], cargs=[])
qc.append(PhaseGate(2.924195801896616), qargs=[qr_qr[13]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[12], qr_qr[3]], cargs=[])
qc.append(PhaseGate(3.5869222695753247), qargs=[qr_qr[7]], cargs=[])
qc.append(CRYGate(0.5425198941315491), qargs=[qr_qr[10], qr_qr[9]], cargs=[])
qc.append(CRZGate(2.6246855324310436), qargs=[qr_qr[9], qr_qr[5]], cargs=[])
qc.append(RZGate(3.5664487201467474), qargs=[qr_qr[10]], cargs=[])
qc.append(C3XGate(5.123156258149011), qargs=[qr_qr[9], qr_qr[8], qr_qr[13], qr_qr[2]], cargs=[])
qc.append(RZGate(4.063125348530302), qargs=[qr_qr[3]], cargs=[])
qc.append(RYGate(5.596839547865737), qargs=[qr_qr[2]], cargs=[])
qc.append(CRYGate(3.107299163630677), qargs=[qr_qr[4], qr_qr[5]], cargs=[])
qc.append(CHGate(), qargs=[qr_qr[14], qr_qr[0]], cargs=[])
qc.append(CRXGate(1.5360054846340931), qargs=[qr_qr[6], qr_qr[11]], cargs=[])
qc.append(PhaseGate(1.8699138522602958), qargs=[qr_qr[6]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[15]], cargs=[])
qc.append(CRZGate(1.4596411939138862), qargs=[qr_qr[3], qr_qr[1]], cargs=[])
qc.append(CU1Gate(2.0594298214588345), qargs=[qr_qr[0], qr_qr[6]], cargs=[])
qc.append(YGate(), qargs=[qr_qr[2]], cargs=[])
qc.append(CUGate(1.736098146392435, 0.3890656207668619, 2.9043239431659917, 1.346317026764786), qargs=[qr_qr[12], qr_qr[13]], cargs=[])
qc.append(TGate(), qargs=[qr_qr[14]], cargs=[])
qc.append(RYGate(4.549942261513976), qargs=[qr_qr[0]], cargs=[])
qc.append(iSwapGate(), qargs=[qr_qr[5], qr_qr[13]], cargs=[])
qc.append(C3XGate(3.5495149923548728), qargs=[qr_qr[12], qr_qr[14], qr_qr[9], qr_qr[7]], cargs=[])
# SECTION
# NAME: OPTIMIZATION_PASSES

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import *
passmanager = PassManager()
qc = passmanager.run(qc)
# SECTION
# NAME: OPTIMIZATION_LEVEL

In [2]:
import os
from qiskit import transpile
import inspect
TRANSPILE_PATH = os.path.abspath(inspect.getfile(transpile))
print(TRANSPILE_PATH)

/home/paltenmo/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/compiler/transpiler.py


In [6]:
content = open(TRANSPILE_PATH, "r").read()
lines = content.split("\n")
for i, line in enumerate(lines):
    print(i, ") ", line)

0 )  # This code is part of Qiskit.
1 )  #
2 )  # (C) Copyright IBM 2017, 2019.
3 )  #
4 )  # This code is licensed under the Apache License, Version 2.0. You may
5 )  # obtain a copy of this license in the LICENSE.txt file in the root directory
6 )  # of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
7 )  #
8 )  # Any modifications or derivative works of this code must retain this
9 )  # copyright notice, and modified files need to carry a notice indicating
10 )  # that they have been altered from the originals.
11 )  
12 )  """Circuit transpile function"""
13 )  import datetime
14 )  import logging
15 )  import warnings
16 )  from time import time
17 )  from typing import List, Union, Dict, Callable, Any, Optional, Tuple, Iterable
18 )  
19 )  from qiskit import user_config
20 )  from qiskit.circuit.quantumcircuit import QuantumCircuit
21 )  from qiskit.circuit.quantumregister import Qubit
22 )  from qiskit.converters import isinstanceint, isinstancelist, dag_to_c

783 )                                  "value": props.duration,
784 )                              }
785 )                          )
786 )                      if props.error is not None:
787 )                          property_list.append(
788 )                              {
789 )                                  "date": datetime.datetime.utcnow(),
790 )                                  "name": "gate_error",
791 )                                  "unit": "",
792 )                                  "value": props.error,
793 )                              }
794 )                          )
795 )                  if property_list:
796 )                      gates.append(
797 )                          {
798 )                              "gate": gate,
799 )                              "qubits": list(qargs),
800 )                              "parameters": property_list,
801 )                              "name": gate + "_".join([str(x) for x in qargs]),
802 )                          }

In [24]:
from utils import load_config_and_check

In [25]:
config = load_config_and_check("../config/qmt_v00.yaml")

In [30]:
selected_gate_set = config["generation_strategy"]["gate_set"]
[eval(g["name"]+"()").name for g in selected_gate_set]

TypeError: __init__() missing 1 required positional argument: 'theta'

from qiskit import transpile
import pdb
pdb.set_trace()
from qiskit import transpile

target_basis = ['rxgate', 'rygate', 'rzgate', 'hgate', 'cxgate']
qc = transpile(qc,
               basis_gates=target_basis, 
               optimization_level=0)

from qiskit import transpile
import pdb
pdb.set_trace()

qc = transpile(qc, basis_gates=['tdgcgate', 'sxdggate', 'chgate', 'rzgate', 'phasegate'], optimization_level=0)
# SECTION
# NAME: MEASUREMENT

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)
/home/paltenmo/projects/ML4Quantum/data/qmt_v00/log_fuzzy.log

In [ ]:
frozenset({('cx', 2), ('u2', 1), ('s', 1), ('u3', 1), ('ch', 2)})
frozenset({('cx', 2), ('s', 1), ('u3', 1), ('ch', 2)})
frozenset({('u1', 1), ('cx', 2), ('u3', 1), ('ch', 2)})
frozenset({('cx', 2), ('u3', 1), ('ch', 2)})
frozenset({('u', 1), ('cx', 2), ('ch', 2)})
frozenset({('u', 1), ('s', 1), ('cx', 2), ('ch', 2)})
frozenset({('p', 1), ('cx', 2), ('u3', 1), ('ch', 2)})
frozenset({('u', 1), ('u1', 1), ('cx', 2), ('ch', 2)})
frozenset({('h', 1), ('cz', 2), ('u3', 1), ('ch', 2)})

In [41]:
sys.version_info.minor

8

In [42]:
!cat /etc/os-release

NAME="Ubuntu"
VERSION="18.04.6 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.6 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


In [32]:
qiskit.__version__

'0.19.1'

In [ ]:
# b /home/paltenmo/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/compiler/transpiler.py:335 
# transpile()

# b /home/paltenmo/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/tools/parallel.py:132 
# parallel_map()

# b /home/paltenmo/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/compiler/transpiler.py:429
# _transpile_circuit()

# b /home/paltenmo/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/transpiler/passmanager.py:218
# run()

# b /home/paltenmo/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/transpiler/passmanager.py:273
# _run_single_circuit()

# <qiskit.transpiler.passes.basis.basis_translator.BasisTranslator object at 0x7fcd39b75160>
# b /home/paltenmo/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/transpiler/runningpassmanager.py:123
# run()

# b /home/paltenmo/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/transpiler/runningpassmanager.py:154
# _do_pass()

# continue for 3 times (the third pass is the problematic: BasisTranslator)

# b /home/paltenmo/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/transpiler/runningpassmanager.py:166
# _run_this_pass()

# b /home/paltenmo/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/transpiler/passes/basis/basis_translator.py:165
# run()

# b /home/paltenmo/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/transpiler/passes/basis/basis_translator.py:367 
# _basis_search()

In [3]:
qc.measure(qr_qr, cr_qr)
# SECTION
# NAME: EXECUTION
from qiskit import Aer, transpile, execute
backend_efb784840b7d41c9b4f9139fn73da3c40 = Aer.get_backend('aer_simulator')
RESULT = execute(qc, backend=backend_efb784840b7d41c9b4f9139f73da3c40,
    shots=44340).result().get_counts(qc)

In [4]:
RESULT

{'0000000000010101': 2,
 '1010000000010101': 702,
 '1000000000000101': 2263,
 '1000000000000100': 1556,
 '0000000000000101': 17306,
 '0010000000010101': 5489,
 '1010000000010100': 528,
 '0010000000010100': 4032,
 '0000000000000100': 12462}

In [6]:

# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *

# SECTION
# NAME: CIRCUIT

qr_qr = QuantumRegister(16, name='qr_qr')
cr_qr = ClassicalRegister(16, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(C3XGate(3.907524998275438), qargs=[qr_qr[11], qr_qr[8], qr_qr[7], qr_qr[0]], cargs=[])
qc.append(RC3XGate(), qargs=[qr_qr[8], qr_qr[15], qr_qr[5], qr_qr[14]], cargs=[])
qc.append(CHGate(), qargs=[qr_qr[10], qr_qr[15]], cargs=[])
qc.append(CU1Gate(4.818142106647685), qargs=[qr_qr[0], qr_qr[6]], cargs=[])
qc.append(RYGate(1.0309627158294603), qargs=[qr_qr[4]], cargs=[])
qc.append(CU1Gate(5.752429763184833), qargs=[qr_qr[8], qr_qr[12]], cargs=[])
qc.append(CRZGate(2.729319301936734), qargs=[qr_qr[15], qr_qr[7]], cargs=[])
qc.append(CUGate(1.7865204421912357,2.217990729902729,5.83693248435634,2.9917879555121014), qargs=[qr_qr[1], qr_qr[0]], cargs=[])
qc.append(PhaseGate(2.924195801896616), qargs=[qr_qr[13]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[12], qr_qr[3]], cargs=[])
qc.append(PhaseGate(3.5869222695753247), qargs=[qr_qr[7]], cargs=[])
qc.append(CRYGate(0.5425198941315491), qargs=[qr_qr[10], qr_qr[9]], cargs=[])
qc.append(CRZGate(2.6246855324310436), qargs=[qr_qr[9], qr_qr[5]], cargs=[])
qc.append(RZGate(3.5664487201467474), qargs=[qr_qr[10]], cargs=[])
qc.append(C3XGate(5.123156258149011), qargs=[qr_qr[9], qr_qr[8], qr_qr[13], qr_qr[2]], cargs=[])
qc.append(RZGate(4.063125348530302), qargs=[qr_qr[3]], cargs=[])
qc.append(RYGate(5.596839547865737), qargs=[qr_qr[2]], cargs=[])
qc.append(CRYGate(3.107299163630677), qargs=[qr_qr[4], qr_qr[5]], cargs=[])
qc.append(CHGate(), qargs=[qr_qr[14], qr_qr[0]], cargs=[])
qc.append(CRXGate(1.5360054846340931), qargs=[qr_qr[6], qr_qr[11]], cargs=[])
qc.append(PhaseGate(1.8699138522602958), qargs=[qr_qr[6]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[15]], cargs=[])
qc.append(CRZGate(1.4596411939138862), qargs=[qr_qr[3], qr_qr[1]], cargs=[])
qc.append(CU1Gate(2.0594298214588345), qargs=[qr_qr[0], qr_qr[6]], cargs=[])
qc.append(YGate(), qargs=[qr_qr[2]], cargs=[])
qc.append(CUGate(1.736098146392435,0.3890656207668619,2.9043239431659917,1.346317026764786), qargs=[qr_qr[12], qr_qr[13]], cargs=[])
qc.append(TGate(), qargs=[qr_qr[14]], cargs=[])
qc.append(RYGate(4.549942261513976), qargs=[qr_qr[0]], cargs=[])
qc.append(iSwapGate(), qargs=[qr_qr[5], qr_qr[13]], cargs=[])
qc.append(C3XGate(3.5495149923548728), qargs=[qr_qr[12], qr_qr[14], qr_qr[9], qr_qr[7]], cargs=[])

# SECTION
# NAME: OPTIMIZATION_PASSES

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import *
passmanager = PassManager()
qc = passmanager.run(qc)

# SECTION
# NAME: OPTIMIZATION_LEVEL

from qiskit import transpile
qc = transpile(qc, basis_gates=['tdggate', 'sxdggate', 'chgate', 'rzgate', 'phasegate'], optimization_level=0)

# SECTION
# NAME: MEASUREMENT

qc.measure(qr_qr, cr_qr)

# SECTION
# NAME: EXECUTION

from qiskit import Aer, transpile, execute
backend_efb784840b7d41c9b4f9139f73da3c40 = Aer.get_backend('qasm_simulator')
RESULT = execute(qc, backend=backend_efb784840b7d41c9b4f9139f73da3c40, shots=44340).result().get_counts(qc)

KeyboardInterrupt: 

In [ ]:
import qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile, execute

circ = QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(0, 2)
circ.measure_all()

decomposed = transpile(circ,
                       basis_gates=['rx', 'ry', 'rz', 'h', 'cx'])  
backend = Aer.get_backend('qasm_simulator')
count = execute(decomposed, backend=backend, shots=1024).result().get_counts(decomposed)

In [36]:
count

{'000': 495, '111': 529}

import qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile, execute

circ = QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(0, 2)
circ.measure_all()

decomposed = transpile(circ,
                       basis_gates=['rxgate', 'rygate', 'rzgate', 'hgate', 'cxgate'])  
backend = Aer.get_backend('qasm_simulator')
count = execute(decomposed, backend=backend, shots=1024).result().get_counts(decomposed)

import qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile, execute

circ = QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(0, 2)
circ.measure_all()

decomposed = transpile(circ,
                       basis_gates=['rxgate', 'rygate', 'rzgate', 'hgate', 'cxgate'])  
backend = Aer.get_backend('qasm_simulator')
count = execute(decomposed, backend=backend, shots=1024).result().get_counts(decomposed)